# Function calling with Bing Search

In this notebook, we'll show how you can use the [Bing Search APIs](https://www.microsoft.com/bing/apis/llm) and [function calling](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling?tabs=python) to ground Azure OpenAI models on data from the web. This is a great way to give the model access to up to date data from the web.

You'll need to create a [Bing Search resouce](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) before you begin.

In [1]:
import requests
import json  
import openai  

# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)
    


# Configure OpenAI environment variables
openai.api_key = config_details['OPENAI_API_KEY']
openai.api_base = config_details['OPENAI_API_BASE']
openai.api_type = "azure"  
openai.api_version = config_details['OPENAI_API_VERSION']

deployment_name = config_details['DEPLOYMENT_NAME'] # You need to use the 0613 version or newer of gpt-35-turbo or gpt-4 to work with functions

bing_search_subscription_key = config_details['BING_SEARCH_SUBSCRIPTION_KEY']
bing_search_url = "https://api.bing.microsoft.com/v7.0/search"

## 1.0 Define a function to call the Bing Search APIs

 To learn more about using the Bing Search APIs with Azure OpenAI, see [Bing Search APIs, with your LLM](https://learn.microsoft.com/bing/search-apis/bing-web-search/use-display-requirements-llm).

In [4]:
def search(query):
    headers = {"Ocp-Apim-Subscription-Key": bing_search_subscription_key}
    params = {"q": query, "textDecorations": False }
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results['webPages']['value']:
        output.append({
            'title': result['name'],
            'link': result['url'],
            'snippet': result['snippet']
        })

    return json.dumps(output)


In [5]:
search("where will the 2032 olymbics be held?")

'[{"title": "2032 Summer Olympics - Wikipedia", "link": "https://en.wikipedia.org/wiki/2032_Summer_Olympics", "snippet": "The 2032 Summer Olympics, officially known as the Games of the XXXV Olympiad and also known as Brisbane 2032 ( Yagara: Meanjin 2032 ), [1] is an upcoming international multi-sport event scheduled to take place between 23 July to 8 August 2032, in Brisbane, Queensland, Australia. [2]"}, {"title": "Venues of the 2032 Summer Olympics and Paralympics", "link": "https://en.wikipedia.org/wiki/Venues_of_the_2032_Summer_Olympics_and_Paralympics", "snippet": "Stadium Australia North Queensland Stadium Barlow Park Toowoomba Sports Ground class=notpageimage| 2032 Olympic and Paralympic venues outside South East Queensland Sporting venues The Gabba Brisbane Convention & Exhibition Centre South Bank Piazza Anna Meares Velodrome Sleeman BMX SuperCross Track Brisbane Aquatics Centre Barrambin / Victoria Park"}, {"title": "The Next Olympics Location: Every Host City Through 2032 | 

## 2.0 Test function calling

In [2]:
system_message = """You are an assistant designed to help people answer questions.

You have access to query the web using Bing Search. You should call bing search whenever a question requires up to date information or could benefit from web data.
"""

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": "How tall is mount rainier?"}]

                
functions = [  
    {
        "name": "search_bing",
        "description": "Searches bing to get up to date information from the web",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query",
                }
            },
            "required": ["query"],
        },
    }
]

response = openai.ChatCompletion.create(
    deployment_id=deployment_name, # You need to use the 0613 version or newer of gpt-35-turbo or gpt-4 to work with functions
    messages=messages,
    functions=functions,
    function_call="auto", 
)

print(response['choices'][0]['message'])

{
  "role": "assistant",
  "function_call": {
    "name": "search_bing",
    "arguments": "{\n  \"query\": \"height of mount rainier\"\n}"
  }
}


## 3.0 Get things running end to end

In [9]:
def run_multiturn_conversation(messages, functions, available_functions, deployment_name):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_name,
        messages=messages,
        functions=functions,
        function_call="auto", 
        temperature=0
    )

    # Step 2: check if GPT wanted to call a function
    while response["choices"][0]["finish_reason"] == 'function_call':
        response_message = response["choices"][0]["message"]
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]  
        
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(**function_args)
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "function_call": {
                    "name": response_message["function_call"]["name"],
                    "arguments": response_message["function_call"]["arguments"],
                },
                "content": None
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in next request:")
        for message in messages:
            print(message)
        print()

        response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_name,
            function_call="auto",
            functions=functions,
            temperature=0
        )  # get a new response from GPT where it can see the function response

    return response

In [10]:
system_message = """You are an assistant designed to help people answer questions.

You have access to query the web using Bing Search. You should call bing search whenever a question requires up to date information or could benefit from web data.
"""

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": "How tall is mount rainier?"}]


available_functions = {'search_bing': search}

result = run_multiturn_conversation(messages, functions, available_functions, deployment_name)

print("Final response:")
print(result['choices'][0]['message']['content'])

Recommended Function call:
{
  "name": "search_bing",
  "arguments": "{\n  \"query\": \"height of Mount Rainier\"\n}"
}

Output of function call:
[{"title": "Mount Rainier - Wikipedia", "link": "https://en.wikipedia.org/wiki/Mount_Rainier", "snippet": "Coordinates: 46\u00b051\u203210\u2033N 121\u00b045\u203238\u2033W Mount Rainier seen from the International Space Station Mount Rainier ( / re\u026a\u02c8n\u026a\u0259r / ray-NEER ), also known as Tahoma, is a large active stratovolcano in the Cascade Range of the Pacific Northwest in the United States."}, {"title": "Geology and History Summary for Mount Rainier - USGS.gov", "link": "https://www.usgs.gov/volcanoes/mount-rainier/geology-and-history-summary-mount-rainier", "snippet": "Public domain.) Mount Rainier is an active volcano of the Cascade Range in Washington State, 50-70 km (30-44 mi) southeast of the Seattle\u2013Tacoma metropolitan area. Volcanism occurs at Mount Rainier and other Cascades arc volcanoes because of the subducti

## Next steps

The example above shows a simple pattern for how you can use function calling to ground Azure OpenAI models on data from the web. Here are some ideas for how you could extend this example:

- Teach the model to cite its sources using prompt engineering
- Define a second function to click into the top search result and extract relevant details from the page. To limit the length of text from the website, you could consider using a separate prompt to summarize the text relevant to the user's query before adding it to the conversation history
- Integrate your own data sources using additional functions